In [ ]:
import os 
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES']= '0'

import numpy as np
import cv2

import pandas as pd
from glob import glob

In [ ]:
from tensorflow import keras
from tensorflow.keras.applications.densenet import DenseNet201

from tensorflow.keras.preprocessing import image

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau

from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Flatten, GlobalAveragePooling2D, AveragePooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.models import model_from_json

# 

## DATA LOAD

In [ ]:
# 이미지 저장 경로
img_list = glob('./data/*/*.*')
data_df=pd.DataFrame({"image": img_list,
                      "label": list(map(lambda x : x.split("/")[-2], img_list)),
                      "is_train": True
                      })

In [ ]:
# train/test split

labels = data_df['label'].unique()
n_split = 0.1

for label in labels:
    n_data = len(data_df[data_df['label']==label])
    data_df.loc[data_df[data_df['label']==label].sample(int(n_data*n_split)).index,'is_train'] = False
    
    print(label, n_data, int(n_data*n_split))
    
# train / test dataset split
train_df = data_df.loc[data_df['is_train']==True].drop(['is_train'],axis=1).reset_index(drop=True)
test_df = data_df.loc[data_df['is_train']==False].drop(['is_train'],axis=1).reset_index(drop=True)
data_df.to_csv('./dataset.csv')

In [ ]:
from generator_v1 import BalencedDataGenerator

data_config ={
    'batch_size' : 32,
    'num_class' : 4,
    'img_h' : 224,
    'img_w' : 224
}

train_generator=BalencedDataGenerator(data=train_df, config=data_config, is_train=True)
valid_generator=BalencedDataGenerator(data=test_df, config=data_config, is_train=False)

# 

## MODEL

In [ ]:
# DenseNet Model
base_model = DenseNet201(include_top=False,weights='imagenet',pooling='avg',input_shape=(224,224,3))

In [ ]:
# Model output 수정
base_model.trainable = True

model=Sequential()
model.add(base_model)

model.add(Dropout(rate=0.5))
model.add(Dense(4, activation='softmax', name="output"))

# Optimizer
adam=Adam()

        
 # Compile
model.compile(optimizer=adam,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Summary
model.summary()

In [ ]:
# Set Callbacks
ckpt=ModelCheckpoint('./logs/epoch-{epoch:02d}-{val_accuracy:.5f}.h5', 
                     monitor='val_accuracy', verbose=1, save_best_only=True)

reducer1=ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3, verbose=1,
                          mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)

reducer2=ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=5, verbose=1,
                          mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)

stopper=EarlyStopping(monitor='val_accuracy', min_delta=0, patience=10, verbose=1,
                      mode='max', baseline=None, restore_best_weights=True)

callbacks_list=[ckpt, stopper, reducer1, reducer2]

In [ ]:
# Train
train_count=train_df.shape[0]
one_epoch=int(train_count/data_config['batch_size'])
num_epochs=300


history=model.fit(train_generator,
                  steps_per_epoch=one_epoch,
                  epochs=num_epochs,
                  callbacks=callbacks_list,
                  initial_epoch=0,
                  verbose=1,
                  validation_data=valid_generator,
                  validation_steps=test_df.shape[0]//data_config['batch_size'])

model.save(os.path.join('./logs', 'train_best.h5'), include_optimizer = False)